In [99]:
import ipywidgets as widgets
import matplotlib.pyplot as plt
#import mpld3
import numpy as np
import sympy as sym

import models
import payoffs
import plotting
import selection_functions
import symbolics

In [100]:
%matplotlib inline
plt.style.use("ggplot")

# One-locus model

Start from full model. Set $x_2=1-x_1, x_3=x_4=0$.

In [101]:
x1 = sym.symbols('x1', real=True, positive=True)
T, R, P, S = sym.symbols('T, R, P, S', real=True, positive=True)
epsilon = sym.symbols("epsilon", real=True, positive=True)

UGA = symbolics.UGA
UgA = symbolics.UgA

# Total offspring (fitness)

In [102]:
x = np.array([[x1], [1 - x1], [0], [0]])
payoff_kernel = np.array([[R, S], [T, P]])
W = models.generalized_sexual_selection(x, UGA, UgA, payoff_kernel, mutation_rate=epsilon)
N, = models.total_offspring(W, x)

In [103]:
N = sym.simplify(N)

In [104]:
N

2.0*P*UGA(x1)**2 - 4.0*P*UGA(x1) + 2.0*P + 2.0*R*UGA(x1)**2 - 2.0*S*UGA(x1)**2 + 2.0*S*UGA(x1) - 2.0*T*UGA(x1)**2 + 2.0*T*UGA(x1)

After a bit of algebra we arrive at the following expression for total offspring.

\begin{align}
   % N(x_1) =& 2.0*P*UGA(x1)**2 - 4.0*P*UGA(x1) + 2.0*P + 2.0*R*UGA(x1)**2 - 2.0*S*UGA(x1)**2 + 2.0*S*UGA(x1) - 2.0*T*UGA(x1)**2 + 2.0*T*UGA(x1)\\
   % N(x_1) =& 2\bigg(PU_{GA}(x_1)^2 - 2PU_{GA}(x_1) + P + RU_{GA}(x_1)^2 - SU_{GA}(x_1)^2 + SU_{GA}(x_1) - TU_{GA}(x_1)^2 + TU_{GA}(x_1)\bigg) \\
   % N(x_1) =& 2\bigg(RU_{GA}(x_1)^2 + PU_{GA}(x_1)^2 - TU_{GA}(x_1)^2 - SU_{GA}(x_1)^2 + SU_{GA}(x_1) + TU_{GA}(x_1) - 2PU_{GA}(x_1) + P\bigg) \\
    N(x_1) =& 2\bigg(\big((R + P) - (T + S)\big)U_{GA}(x_1)^2 + \big((T + S) - 2P\big)U_{GA}(x_1) + P\bigg) \\
\end{align}

## Equilibrium total offspring

Consider an equilibrium $x^*$ and define the constant equilibrium selection probability as $U_{GA}^* = U_{GA}(x^*)$.

In [105]:
UGA_star = sym.symbols("UGA_star", real=True, nonnegative=True)
equilibrium_total_offspring = N.subs({UGA(x1): UGA_star})

In [106]:
equilibrium_total_offspring

2.0*P*UGA_star**2 - 4.0*P*UGA_star + 2.0*P + 2.0*R*UGA_star**2 - 2.0*S*UGA_star**2 + 2.0*S*UGA_star - 2.0*T*UGA_star**2 + 2.0*T*UGA_star

In [107]:
# solve for the interior value of UGA* that maximizes total offspring
foc = equilibrium_total_offspring.diff(UGA_star, 1)
optimal_UGA_star, = sym.solve(foc, UGA_star)

In [108]:
optimal_UGA_star

(P - 0.5*S - 0.5*T)/(P + R - S - T)

In [12]:
_equilibrium_total_offspring = sym.lambdify((UGA_star, T, R, P, S),
                                            equilibrium_total_offspring,
                                            modules="numpy")

def plot_total_offspring(T, R, P, S):
    fig, ax = plt.subplots(1, 1, figsize=(10,6))
    us = np.linspace(0, 1, 100)
    ax.plot(us, _equilibrium_total_offspring(us, T, R, P, S))
    ax.set_title(r"Total number of offspring", fontsize=25)
    ax.set_xlabel(r"$U_{GA}^*$", fontsize=25)
    ax.set_ylabel(r"N", rotation="horizontal", fontsize=25)
    plt.show()

In [14]:
# sliders used to control the Prisoner's Dilemma Payoffs
T_slider = widgets.FloatSlider(value=25, min=0, max=100, step=0.1, description=r"$T$")
R_slider = widgets.FloatSlider(value=3, min=0, max=100, step=0.1, description=r"$R$")
P_slider = widgets.FloatSlider(value=2, min=0, max=100, step=0.1, description=r"$P$")
S_slider = widgets.FloatSlider(value=1, min=0, max=100, step=0.1, description=r"$S$")

w = widgets.interactive(plot_total_offspring, T=T_slider, R=R_slider, P=P_slider, S=S_slider)
display(w)

interactive(children=(FloatSlider(value=25.0, description='$T$'), FloatSlider(value=3.0, description='$R$'), F…

# The locus of potential equilibria (LPE)

In [109]:
(f,), *_ = models.offspring_genotypes_evolution(W, x)

In [110]:
f

-x1 + (x1*((-epsilon + 1)*(2*R*UGA(x1) + 2*S*(-UGA(x1) + 1))*UGA(x1) + 0.5*(2*P*(-UGA(x1) + 1) + 2*T*UGA(x1))*(-UGA(x1) + 1)) + (-x1 + 1)*(epsilon*(2*P*(-UGA(x1) + 1) + 2*T*UGA(x1))*(-UGA(x1) + 1) + 0.5*(2*R*UGA(x1) + 2*S*(-UGA(x1) + 1))*UGA(x1)))/(x1*(epsilon*(2*R*UGA(x1) + 2*S*(-UGA(x1) + 1))*UGA(x1) + 0.5*(2*P*(-UGA(x1) + 1) + 2*T*UGA(x1))*(-UGA(x1) + 1)) + x1*((-epsilon + 1)*(2*R*UGA(x1) + 2*S*(-UGA(x1) + 1))*UGA(x1) + 0.5*(2*P*(-UGA(x1) + 1) + 2*T*UGA(x1))*(-UGA(x1) + 1)) + (-x1 + 1)*(epsilon*(2*P*(-UGA(x1) + 1) + 2*T*UGA(x1))*(-UGA(x1) + 1) + 0.5*(2*R*UGA(x1) + 2*S*(-UGA(x1) + 1))*UGA(x1)) + (-x1 + 1)*((-epsilon + 1)*(2*P*(-UGA(x1) + 1) + 2*T*UGA(x1))*(-UGA(x1) + 1) + 0.5*(2*R*UGA(x1) + 2*S*(-UGA(x1) + 1))*UGA(x1)))

In [40]:
n, d = sym.fraction(f + x1)

In [43]:
n

x1*((-epsilon + 1)*(2*R*UGA(x1) + 2*S*(-UGA(x1) + 1))*UGA(x1) + 0.5*(2*P*(-UGA(x1) + 1) + 2*T*UGA(x1))*(-UGA(x1) + 1)) + (-x1 + 1)*(epsilon*(2*P*(-UGA(x1) + 1) + 2*T*UGA(x1))*(-UGA(x1) + 1) + 0.5*(2*R*UGA(x1) + 2*S*(-UGA(x1) + 1))*UGA(x1))

In [ ]:
x1*((-epsilon + 1)*(2*R*UGA(x1) + 2*S*(-UGA(x1) + 1))*UGA(x1) + 0.5*(2*P*(-UGA(x1) + 1) + 2*T*UGA(x1))*(-UGA(x1) + 1))

In [ ]:
(-x1 + 1)*(epsilon*(2*P*(-UGA(x1) + 1) + 2*T*UGA(x1))*(-UGA(x1) + 1) + 0.5*(2*R*UGA(x1) + 2*S*(-UGA(x1) + 1))*UGA(x1))

In [42]:
sym.expand(d)

2*P*UGA(x1)**2 - 4*P*UGA(x1) + 2*P + 2.0*R*UGA(x1)**2 - 2.0*S*UGA(x1)**2 + 2.0*S*UGA(x1) - 2*T*UGA(x1)**2 + 2*T*UGA(x1)

In [30]:
n

x1*(2*(-epsilon + 1)*(R*UGA(x1) + S*(-UGA(x1) + 1))*UGA(x1) + 1.0*(P*(-UGA(x1) + 1) + T*UGA(x1))*(-UGA(x1) + 1)) - x1*(x1*(2*epsilon*(R*UGA(x1) + S*(-UGA(x1) + 1))*UGA(x1) + 1.0*(P*(-UGA(x1) + 1) + T*UGA(x1))*(-UGA(x1) + 1)) + x1*(2*(-epsilon + 1)*(R*UGA(x1) + S*(-UGA(x1) + 1))*UGA(x1) + 1.0*(P*(-UGA(x1) + 1) + T*UGA(x1))*(-UGA(x1) + 1)) + (-x1 + 1)*(2*epsilon*(P*(-UGA(x1) + 1) + T*UGA(x1))*(-UGA(x1) + 1) + 1.0*(R*UGA(x1) + S*(-UGA(x1) + 1))*UGA(x1)) + (-x1 + 1)*(2*(-epsilon + 1)*(P*(-UGA(x1) + 1) + T*UGA(x1))*(-UGA(x1) + 1) + 1.0*(R*UGA(x1) + S*(-UGA(x1) + 1))*UGA(x1))) + (-x1 + 1)*(2*epsilon*(P*(-UGA(x1) + 1) + T*UGA(x1))*(-UGA(x1) + 1) + 1.0*(R*UGA(x1) + S*(-UGA(x1) + 1))*UGA(x1))

\begin{align}
    x_1(2R(-\epsilon + 1)U(x_1)^2 + 2S(-\epsilon + 1)U(x_1)(-U(x_1) + 1) + P(-U(x_1) + 1)^2 + TU(x_1)(-U(x_1) + 1)) - x_1(x_1(2\epsilon(RU(x_1) + S(-U(x_1) + 1))U(x_1) + (P(-U(x_1) + 1) + TU(x_1))(-U(x_1) + 1)) + x_1(2(-\epsilon + 1)(RU(x_1) + S(-U(x_1) + 1))U(x_1) + (P(-U(x_1) + 1) + TU(x_1))(-U(x_1) + 1)) + (-x_1 + 1)(2\epsilon(P(-U(x_1) + 1) + TU(x_1))(-U(x_1) + 1) + (RU(x_1) + S(-U(x_1) + 1))U(x_1)) + (-x_1 + 1)(2(-\epsilon + 1)(P(-U(x_1) + 1) + TU(x_1))(-U(x_1) + 1) + (RU(x_1) + S(-U(x_1) + 1))U(x_1))) + (-x_1 + 1)(2\epsilon(P(-U(x_1) + 1) + TU(x_1))(-U(x_1) + 1) + (RU(x_1) + S(-U(x_1) + 1))U(x_1))
\end{align}

In [29]:
# denominator of the above expression is just total offspring, N(x1)
sym.expand(d)

2*P*UGA(x1)**2 - 4*P*UGA(x1) + 2*P + 2.0*R*UGA(x1)**2 - 2.0*S*UGA(x1)**2 + 2.0*S*UGA(x1) - 2*T*UGA(x1)**2 + 2*T*UGA(x1)

In [19]:
sym.together(sym.expand(f))

(-2*P*epsilon*x1*UGA(x1)**2 + 4*P*epsilon*x1*UGA(x1) - 2*P*epsilon*x1 + 2*P*epsilon*UGA(x1)**2 - 4*P*epsilon*UGA(x1) + 2*P*epsilon + 1.0*P*x1*UGA(x1)**2 - 2.0*P*x1*UGA(x1) + 1.0*P*x1 - 2*R*epsilon*x1*UGA(x1)**2 + 1.0*R*x1*UGA(x1)**2 + 1.0*R*UGA(x1)**2 + 2*S*epsilon*x1*UGA(x1)**2 - 2*S*epsilon*x1*UGA(x1) - 1.0*S*x1*UGA(x1)**2 + 1.0*S*x1*UGA(x1) - 1.0*S*UGA(x1)**2 + 1.0*S*UGA(x1) + 2*T*epsilon*x1*UGA(x1)**2 - 2*T*epsilon*x1*UGA(x1) - 2*T*epsilon*UGA(x1)**2 + 2*T*epsilon*UGA(x1) - 1.0*T*x1*UGA(x1)**2 + 1.0*T*x1*UGA(x1) - x1*(2*P*UGA(x1)**2 - 4*P*UGA(x1) + 2*P + 2.0*R*UGA(x1)**2 - 2.0*S*UGA(x1)**2 + 2.0*S*UGA(x1) - 2*T*UGA(x1)**2 + 2*T*UGA(x1)))/(2*P*UGA(x1)**2 - 4*P*UGA(x1) + 2*P + 2.0*R*UGA(x1)**2 - 2.0*S*UGA(x1)**2 + 2.0*S*UGA(x1) - 2*T*UGA(x1)**2 + 2*T*UGA(x1))

\begin{align}
    %\dot{x}_1 =& \frac{(-2P\epsilon x_1U(x_1)^2 + 4P\epsilon x_1U(x_1) - 2P\epsilon x_1 + 2P\epsilon U(x_1)^2 - 4P\epsilon U(x_1) + 2P\epsilon + Px_1U(x_1)^2 - 2Px_1U(x_1) + Px_1 - 2R\epsilon x_1U(x_1)^2 + Rx_1U(x_1)^2 + RU(x_1)^2 + 2S\epsilon x_1U(x_1)^2 - 2S\epsilon x_1U(x_1) - Sx_1U(x_1)^2 + Sx_1U(x_1) - SU(x_1)^2 + SU(x_1) + 2T\epsilon x_1U(x_1)^2 - 2T\epsilon x_1U(x_1) - 2T\epsilon U(x_1)^2 + 2T\epsilon U(x_1) - Tx_1U(x_1)^2 + Tx_1U(x_1) - x_1(2PU(x_1)^2 - 4PU(x_1) + 2P + 2RU(x_1)^2 - 2SU(x_1)^2 + 2SU(x_1) - 2TU(x_1)^2 + 2TU(x_1)))}{(2PU(x_1)^2 - 4PU(x_1) + 2P + 2RU(x_1)^2 - 2SU(x_1)^2 + 2SU(x_1) - 2TU(x_1)^2 + 2TU(x_1))} \\
    %\dot{x}_1 =& \frac{(-2P\epsilon x_1U(x_1)^2 + 4P\epsilon x_1U(x_1) - 2P\epsilon x_1 + 2P\epsilon U(x_1)^2 - 4P\epsilon U(x_1) + 2P\epsilon + Px_1U(x_1)^2 - 2Px_1U(x_1) + Px_1 - 2R\epsilon x_1U(x_1)^2 + Rx_1U(x_1)^2 + RU(x_1)^2 + 2S\epsilon x_1U(x_1)^2 - 2S\epsilon x_1U(x_1) - Sx_1U(x_1)^2 + Sx_1U(x_1) - SU(x_1)^2 + SU(x_1) + 2T\epsilon x_1U(x_1)^2 - 2T\epsilon x_1U(x_1) - 2T\epsilon U(x_1)^2 + 2T\epsilon U(x_1) - Tx_1U(x_1)^2 + Tx_1U(x_1))}{(2PU(x_1)^2 - 4PU(x_1) + 2P + 2RU(x_1)^2 - 2SU(x_1)^2 + 2SU(x_1) - 2TU(x_1)^2 + 2TU(x_1))} - x_1 \\
    %\dot{x}_1 =& \frac{(-2P\epsilon x_1U(x_1)^2 + 4P\epsilon x_1U(x_1) - 2P\epsilon x_1 - 2R\epsilon x_1U(x_1)^2 + 2S\epsilon x_1U(x_1)^2 - 2S\epsilon x_1U(x_1) + 2T\epsilon x_1U(x_1)^2 - 2T\epsilon x_1U(x_1) + 2P\epsilon U(x_1)^2 - 4P\epsilon U(x_1) + 2P\epsilon + Px_1U(x_1)^2 - 2Px_1U(x_1) + Px_1 + Rx_1U(x_1)^2 + RU(x_1)^2 - Sx_1U(x_1)^2 + Sx_1U(x_1) - SU(x_1)^2 + SU(x_1) - 2T\epsilon U(x_1)^2 + 2T\epsilon U(x_1) - Tx_1U(x_1)^2 + Tx_1U(x_1))}{(2PU(x_1)^2 - 4PU(x_1) + 2P + 2RU(x_1)^2 - 2SU(x_1)^2 + 2SU(x_1) - 2TU(x_1)^2 + 2TU(x_1))} - x_1
    \dot{x}_1 =& \frac{(-\epsilon x_1\big(2PU(x_1)^2 - 4PU(x_1) + 2P + 2RU(x_1)^2 - 2SU(x_1)^2 + 2SU(x_1) - 2TU(x_1)^2 + 2TU(x_1)\big) + 2P\epsilon U(x_1)^2 - 4P\epsilon U(x_1) + 2P\epsilon + Px_1U(x_1)^2 - 2Px_1U(x_1) + Px_1 + Rx_1U(x_1)^2 + RU(x_1)^2 - Sx_1U(x_1)^2 + Sx_1U(x_1) - SU(x_1)^2 + SU(x_1) - 2T\epsilon U(x_1)^2 + 2T\epsilon U(x_1) - Tx_1U(x_1)^2 + Tx_1U(x_1))}{(2PU(x_1)^2 - 4PU(x_1) + 2P + 2RU(x_1)^2 - 2SU(x_1)^2 + 2SU(x_1) - 2TU(x_1)^2 + 2TU(x_1))} - x_1 \\
    \dot{x}_1 =& \frac{2P\epsilon U(x_1)^2 - 4P\epsilon U(x_1) + 2P\epsilon + Px_1U(x_1)^2 - 2Px_1U(x_1) + Px_1 + Rx_1U(x_1)^2 + RU(x_1)^2 - Sx_1U(x_1)^2 + Sx_1U(x_1) - SU(x_1)^2 + SU(x_1) - 2T\epsilon U(x_1)^2 + 2T\epsilon U(x_1) - Tx_1U(x_1)^2 + Tx_1U(x_1)}{(2PU(x_1)^2 - 4PU(x_1) + 2P + 2RU(x_1)^2 - 2SU(x_1)^2 + 2SU(x_1) - 2TU(x_1)^2 + 2TU(x_1))} - (1 + \epsilon)x_1 \\
    \dot{x}_1 =& \frac{2P\epsilon U(x_1)^2 - 4P\epsilon U(x_1) + 2P\epsilon + Px_1U(x_1)^2 - 2Px_1U(x_1) + Px_1 + Rx_1U(x_1)^2 + RU(x_1)^2 - Sx_1U(x_1)^2 + Sx_1U(x_1) - SU(x_1)^2 + SU(x_1) - 2T\epsilon U(x_1)^2 + 2T\epsilon U(x_1) - Tx_1U(x_1)^2 + Tx_1U(x_1)}{N(x_1)} - (1 + \epsilon)x_1
\end{align}

In [24]:
sym.expand((T + S) * UGA(x1) * (1 - UGA(x1)) * x1 * (1 - x1))

S*x1**2*UGA(x1)**2 - S*x1**2*UGA(x1) - S*x1*UGA(x1)**2 + S*x1*UGA(x1) + T*x1**2*UGA(x1)**2 - T*x1**2*UGA(x1) - T*x1*UGA(x1)**2 + T*x1*UGA(x1)

In [13]:
# solving for the LPE using Python
x1_star, = sym.solve(f, x1, implicit=True)
x1_star =  sym.cancel(x1_star.subs({UGA(x1): UGA_star}))

In [14]:
sym.factor(x1_star)

1.0*(0.5*P*UGA_star**2*epsilon - 1.0*P*UGA_star*epsilon + 0.5*P*epsilon + 0.25*R*UGA_star**2 - 0.25*S*UGA_star**2 + 0.25*S*UGA_star - 0.5*T*UGA_star**2*epsilon + 0.5*T*UGA_star*epsilon)/((1.0*epsilon + 0.5)*(0.5*P*UGA_star**2 - 1.0*P*UGA_star + 0.5*P + 0.5*R*UGA_star**2 - 0.5*S*UGA_star**2 + 0.5*S*UGA_star - 0.5*T*UGA_star**2 + 0.5*T*UGA_star))

Setting the equation of motion equal to zero, rearranging the resulting equation yields the locus of potential equilibria (LPE). The LPE represents, for each possible $x_1$, the value that $U_{GA}$ would have to take in order for that $x_1$ to be an equilbrium. 

\begin{align}
  L(x^*) \equiv x_1^* - 2\left(\frac{\big(R - S\big)U_{GA}^{*2} + \big(S - M\big)U_{GA}^* - 2\epsilon\big((T - P)U_{GA}^{*2} - (T - 2P)U_{GA}^* - P\big)}{N(U_{GA}^*)(1 + 2\epsilon)}\right) = 0\\
\end{align}

We already know that $x_1^*=0, U_{GA}^*=0$ and $x_1^*=1, U_{GA}^*=1$ are equilibria, so it follows that $L(0)=0$ and $L(1)=0$.


Differentiate $x_1^*$ with respect to $U_{GA}^*$ yields an expression that will be useful for assessing stability and invadability of an equilibrium.

In [27]:
f

-x1 + (x1*((-epsilon + 1)*(2*R*UGA(x1) + 2*S*(-UGA(x1) + 1))*UGA(x1) + 0.5*(2*P*(-UGA(x1) + 1) + 2*T*UGA(x1))*(-UGA(x1) + 1)) + (-x1 + 1)*(epsilon*(2*P*(-UGA(x1) + 1) + 2*T*UGA(x1))*(-UGA(x1) + 1) + 0.5*(2*R*UGA(x1) + 2*S*(-UGA(x1) + 1))*UGA(x1)))/(x1*(epsilon*(2*R*UGA(x1) + 2*S*(-UGA(x1) + 1))*UGA(x1) + 0.5*(2*P*(-UGA(x1) + 1) + 2*T*UGA(x1))*(-UGA(x1) + 1)) + x1*((-epsilon + 1)*(2*R*UGA(x1) + 2*S*(-UGA(x1) + 1))*UGA(x1) + 0.5*(2*P*(-UGA(x1) + 1) + 2*T*UGA(x1))*(-UGA(x1) + 1)) + (-x1 + 1)*(epsilon*(2*P*(-UGA(x1) + 1) + 2*T*UGA(x1))*(-UGA(x1) + 1) + 0.5*(2*R*UGA(x1) + 2*S*(-UGA(x1) + 1))*UGA(x1)) + (-x1 + 1)*((-epsilon + 1)*(2*P*(-UGA(x1) + 1) + 2*T*UGA(x1))*(-UGA(x1) + 1) + 0.5*(2*R*UGA(x1) + 2*S*(-UGA(x1) + 1))*UGA(x1)))

In [84]:
d = sym.symbols('d', positive=True)
U = selection_functions.kirkpatrick_selection(x1, d)

sym.solve(f.subs({UGA(x1): U, epsilon: 0}), x1)

[1.00000000000000, -(-P + S*d)/(P + R*d**2 - S*d - T*d)]

In [85]:
sym.factor(f.subs({UGA(x1): U}), x1)

-1.5*(-0.333333333333333*P*epsilon + x1**3*(0.333333333333333*P*epsilon + 0.166666666666667*P + 0.333333333333333*R*d**2*epsilon + 0.166666666666667*R*d**2 - 0.333333333333333*S*d*epsilon - 0.166666666666667*S*d - 0.333333333333333*T*d*epsilon - 0.166666666666667*T*d) + x1**2*(-1.0*P*epsilon - 0.333333333333333*P - 0.166666666666667*R*d**2 + 0.333333333333333*S*d*epsilon + 0.333333333333333*S*d + 0.666666666666667*T*d*epsilon + 0.166666666666667*T*d) + x1*(1.0*P*epsilon + 0.166666666666667*P - 0.166666666666667*S*d - 0.333333333333333*T*d*epsilon))/(0.5*P + x1**2*(0.5*P + 0.5*R*d**2 - 0.5*S*d - 0.5*T*d) + x1*(-1.0*P + 0.5*S*d + 0.5*T*d))

In [86]:
n, _ = sym.fraction(sym.factor(f.subs({UGA(x1): U}), x1))

In [87]:
sym.factor(N.subs({UGA(x1): U}), x1)

4.0*(0.5*P + x1**2*(0.5*P + 0.5*R*d**2 - 0.5*S*d - 0.5*T*d) + x1*(-1.0*P + 0.5*S*d + 0.5*T*d))/(x1*(d - 1) + 1)**2

In [88]:
p = sym.Poly(n, x1)

In [98]:
sym.solve(sym.factor(p.discriminant(), epsilon).subs({epsilon**2: 0}), epsilon)

[(-0.25*P**2*R**2*d**2 + 0.5*P**2*R*T*d - 0.25*P**2*T**2 + 0.5*P*R**2*S*d**3 - P*R*S*T*d**2 + 0.5*P*S*T**2*d - 0.25*R**2*S**2*d**4 + 0.5*R*S**2*T*d**3 - 0.25*S**2*T**2*d**2)/(-2.0*P**3*R + 5.0*P**2*R*S*d + P**2*S*T - 2.0*P*R**3*d**4 + 5.0*P*R**2*T*d**3 - 4.0*P*R*S**2*d**2 - 4.0*P*R*T**2*d**2 - 2.0*P*S**2*T*d + P*T**3*d + R**2*S*T*d**4 + R*S**3*d**3 - 2.0*R*S*T**2*d**3 + S**3*T*d**2 + S*T**3*d**2)]

In [92]:
sym.factor(sym.expand(1.6875 *(-1.0*P**2*R**2*d**2 + 0.666666666666667*P*R*S*T*d**2 + 0.148148148148148*P*S**3*d + 0.148148148148148*R*T**3*d**3 + 0.037037037037037*S**2*T**2*d**2)))

1.6875*d*(-1.0*P**2*R**2*d + 0.666666666666667*P*R*S*T*d + 0.148148148148148*P*S**3 + 0.148148148148148*R*T**3*d**2 + 0.037037037037037*S**2*T**2*d)

In [56]:
roots = sym.solve(n, x1)

In [64]:
sym.factor(sym.together(roots[0]), epsilon)

KeyboardInterrupt: 

In [67]:
roots[0]

-0.333333333333333*(-3.0*(6.0*P*epsilon + P - S*d - 2.0*T*d*epsilon)/(2.0*P*epsilon + P + 2.0*R*d**2*epsilon + R*d**2 - 2.0*S*d*epsilon - S*d - 2.0*T*d*epsilon - T*d) + (-6.0*P*epsilon - 2.0*P - R*d**2 + 2.0*S*d*epsilon + 2.0*S*d + 4.0*T*d*epsilon + T*d)**2/(2.0*P*epsilon + P + 2.0*R*d**2*epsilon + R*d**2 - 2.0*S*d*epsilon - S*d - 2.0*T*d*epsilon - T*d)**2)/(-27.0*P*epsilon/(2.0*P*epsilon + P + 2.0*R*d**2*epsilon + R*d**2 - 2.0*S*d*epsilon - S*d - 2.0*T*d*epsilon - T*d) + 0.5*(-4.0*(-3.0*(6.0*P*epsilon + P - S*d - 2.0*T*d*epsilon)/(2.0*P*epsilon + P + 2.0*R*d**2*epsilon + R*d**2 - 2.0*S*d*epsilon - S*d - 2.0*T*d*epsilon - T*d) + (-6.0*P*epsilon - 2.0*P - R*d**2 + 2.0*S*d*epsilon + 2.0*S*d + 4.0*T*d*epsilon + T*d)**2/(2.0*P*epsilon + P + 2.0*R*d**2*epsilon + R*d**2 - 2.0*S*d*epsilon - S*d - 2.0*T*d*epsilon - T*d)**2)**3 + (-54.0*P*epsilon/(2.0*P*epsilon + P + 2.0*R*d**2*epsilon + R*d**2 - 2.0*S*d*epsilon - S*d - 2.0*T*d*epsilon - T*d) - 9.0*(6.0*P*epsilon + P - S*d - 2.0*T*d*epsilon)*(-

In [66]:
sym.expand(roots[1])

(-0.333333333333333*((-27.0*P*epsilon*(2.0*P*epsilon + P + 2.0*R*d**2*epsilon + R*d**2 - 2.0*S*d*epsilon - S*d - 2.0*T*d*epsilon - T*d)**2 + 0.5*(-4.0*(-3.0*(6.0*P*epsilon + P - S*d - 2.0*T*d*epsilon)*(2.0*P*epsilon + P + 2.0*R*d**2*epsilon + R*d**2 - 2.0*S*d*epsilon - S*d - 2.0*T*d*epsilon - T*d) + (-6.0*P*epsilon - 2.0*P - R*d**2 + 2.0*S*d*epsilon + 2.0*S*d + 4.0*T*d*epsilon + T*d)**2)**3 + (-54.0*P*epsilon*(2.0*P*epsilon + P + 2.0*R*d**2*epsilon + R*d**2 - 2.0*S*d*epsilon - S*d - 2.0*T*d*epsilon - T*d)**2 - 9.0*(6.0*P*epsilon + P - S*d - 2.0*T*d*epsilon)*(-6.0*P*epsilon - 2.0*P - R*d**2 + 2.0*S*d*epsilon + 2.0*S*d + 4.0*T*d*epsilon + T*d)*(2.0*P*epsilon + P + 2.0*R*d**2*epsilon + R*d**2 - 2.0*S*d*epsilon - S*d - 2.0*T*d*epsilon - T*d) + 2.0*(-6.0*P*epsilon - 2.0*P - R*d**2 + 2.0*S*d*epsilon + 2.0*S*d + 4.0*T*d*epsilon + T*d)**3)**2)**0.5*(2.0*P*epsilon + P + 2.0*R*d**2*epsilon + R*d**2 - 2.0*S*d*epsilon - S*d - 2.0*T*d*epsilon - T*d)**3*Abs(2.0*P*epsilon + P + 2.0*R*d**2*epsilon + R

In [15]:
dx1star_dUGA_star = sym.factor(sym.cancel(sym.diff(x1_star, UGA_star, 1)))

In [16]:
dx1star_dUGA_star

1.0*(2.0*epsilon - 1.0)*(2.0*P*R*UGA_star**2 - 2.0*P*R*UGA_star - 1.0*P*S*UGA_star**2 + 2.0*P*S*UGA_star - 1.0*P*S - 1.0*R*T*UGA_star**2)/((2.0*epsilon + 1.0)*(1.0*P*UGA_star**2 - 2.0*P*UGA_star + 1.0*P + 1.0*R*UGA_star**2 - 1.0*S*UGA_star**2 + 1.0*S*UGA_star - 1.0*T*UGA_star**2 + 1.0*T*UGA_star)**2)

In [17]:
optimal_x1_star = sym.factor(sym.cancel(x1_star.subs({UGA_star: optimal_UGA_star})))

In [18]:
optimal_x1_star

1.0*(2.0*P**2*R + 4.0*P*R**2*epsilon - 4.0*P*R*S*epsilon - 2.0*P*R*T + 1.0*P*S**2*epsilon - 1.0*P*S**2 - 1.0*P*S*T - 1.0*P*T**2*epsilon - 0.5*R*S**2 - 2.0*R*S*T*epsilon - 2.0*R*T**2*epsilon + 0.5*R*T**2 + 0.5*S**3 + 1.0*S**2*T*epsilon + 1.0*S**2*T + 2.0*S*T**2*epsilon + 0.5*S*T**2 + 1.0*T**3*epsilon)/((1.0*epsilon + 0.5)*(0.5*P + 0.5*R - 0.5*S - 0.5*T)*(2.0*P*R - 0.5*S**2 - 1.0*S*T - 0.5*T**2))

## Interactive plot of the locus of potential equilibria

In [34]:
_locus_of_potential_equilibria = sym.lambdify((UGA_star, T, R, P, S, epsilon),
                                              x1_star,
                                              modules="numpy")


def plot_locus_of_potential_equilibria(selection_function, d1, T, R, P, S, epsilon):
    assert T > R > P > S
    us = np.linspace(0, 1, 100)
    xs = _locus_of_potential_equilibria(us, T, R, P, S, epsilon)

    fig, ax = plt.subplots(1, 1, figsize=(10, 8))
    ax.plot(xs, us, label="LPE")
    ax.set_xlabel(r"$x_1^*$", fontsize=25)
    ax.set_ylabel(r"$U_{GA}^*$", rotation="horizontal", fontsize=25)
    ax.plot(us, us, 'k--')
    
    # create the selection functions
    if selection_function == "kirkpatrick":
        UGA = lambda x_A: selection_functions.kirkpatrick_selection(x_A, d1)
    elif selection_function == "seger":
        UGA = lambda x_A: selection_functions.seger_selection(x_A, d1)
    elif selection_function == "wright":
        UGA = lambda x_A: selection_functions.wright_selection(x_A, d1)
    else:
        valid_funcs = ("kirkpatrick", "seger", "wright")
        msg = "Selection_function must be one of {}, {}, or {}.".format(*valid_funcs)
        raise ValueError(msg)
        
    # add selection functions for comparison
    ax.plot(xs, UGA(xs), label=selection_function)
    ax.legend()
    
    plt.show()

In [35]:
#mpld3.enable_notebook()

# sliders used to control the Prisoner's Dilemma Payoffs
T_slider = widgets.FloatSlider(value=25, min=0, max=100, step=0.1, description=r"$T$")
R_slider = widgets.FloatSlider(value=3, min=0, max=100, step=0.1, description=r"$R$")
P_slider = widgets.FloatSlider(value=2, min=0, max=100, step=0.1, description=r"$P$")
S_slider = widgets.FloatSlider(value=1, min=0, max=100, step=0.1, description=r"$S$")

# slider used to control metabolic costs
e_slider = widgets.FloatSlider(value=0, min=0, max=1, step=1e-3, description=r"$\epsilon$", readout_format=".3f")

# slider used to control which selection function is being used
UGA_slider = widgets.Dropdown(options=["kirkpatrick", "seger", "wright"], index=0, description=r"$U_{\gamma(j)A}$")

# slider that controls the parameters of the selection function
d1_slider = widgets.FloatSlider(value=1.0, min=0.0, max=10, step=0.05, description=r"$d_1$")

w = widgets.interactive(plot_locus_of_potential_equilibria, selection_function=UGA_slider, d1=d1_slider,
                        T=T_slider, R=R_slider, P=P_slider, S=S_slider, epsilon=e_slider)
display(w)

interactive(children=(Dropdown(description='$U_{\\gamma(j)A}$', options=('kirkpatrick', 'seger', 'wright'), va…

# Stability

Necessary and sufficient conditions for stability require that the derivative of the equation of motion with respect to $x_1$ be strictly negative when evaluated at the potential equilibrium.

In [111]:
f_jac = f.diff(x1, 1)

In [112]:
f_jac

(x1*((-epsilon + 1)*(2*R*UGA(x1) + 2*S*(-UGA(x1) + 1))*UGA(x1) + 0.5*(2*P*(-UGA(x1) + 1) + 2*T*UGA(x1))*(-UGA(x1) + 1)) + (-x1 + 1)*(epsilon*(2*P*(-UGA(x1) + 1) + 2*T*UGA(x1))*(-UGA(x1) + 1) + 0.5*(2*R*UGA(x1) + 2*S*(-UGA(x1) + 1))*UGA(x1)))*(epsilon*(2*P*(-UGA(x1) + 1) + 2*T*UGA(x1))*(-UGA(x1) + 1) - epsilon*(2*R*UGA(x1) + 2*S*(-UGA(x1) + 1))*UGA(x1) - x1*(epsilon*(2*R*UGA(x1) + 2*S*(-UGA(x1) + 1))*U_prime(x1) + epsilon*(2*R*U_prime(x1) - 2*S*U_prime(x1))*UGA(x1) - 0.5*(2*P*(-UGA(x1) + 1) + 2*T*UGA(x1))*U_prime(x1) + (-2*P*U_prime(x1) + 2*T*U_prime(x1))*(-0.5*UGA(x1) + 0.5)) - x1*((-epsilon + 1)*(2*R*UGA(x1) + 2*S*(-UGA(x1) + 1))*U_prime(x1) + (-epsilon + 1)*(2*R*U_prime(x1) - 2*S*U_prime(x1))*UGA(x1) - 0.5*(2*P*(-UGA(x1) + 1) + 2*T*UGA(x1))*U_prime(x1) + (-2*P*U_prime(x1) + 2*T*U_prime(x1))*(-0.5*UGA(x1) + 0.5)) + (-epsilon + 1)*(2*P*(-UGA(x1) + 1) + 2*T*UGA(x1))*(-UGA(x1) + 1) - (-epsilon + 1)*(2*R*UGA(x1) + 2*S*(-UGA(x1) + 1))*UGA(x1) - (-x1 + 1)*(-epsilon*(2*P*(-UGA(x1) + 1) + 2*T

In [113]:
UGA_prime_star = sym.symbols("UGA_prime_star", real=True, nonnegative=True)
evaluated_f_jac = f_jac.subs({sym.Derivative(UGA(x1), x1): UGA_prime_star, UGA(x1): UGA_star, x1: x1_star})

In [ ]:
sym.expand(evaluated_f_jac)

In [17]:
numerator, denominator = sym.fraction(sym.factor(sym.together(evaluated_f_jac)))

In [18]:
# denominator is positive
denominator

(0.5*P*UGA_star**2 - 1.0*P*UGA_star + 0.5*P + 0.5*R*UGA_star**2 - 0.5*S*UGA_star**2 + 0.5*S*UGA_star - 0.5*T*UGA_star**2 + 0.5*T*UGA_star)**2

In [24]:
numerator

-0.25*P**2*UGA_star**4*epsilon - 0.125*P**2*UGA_star**4 + 1.0*P**2*UGA_star**3*epsilon + 0.5*P**2*UGA_star**3 - 1.5*P**2*UGA_star**2*epsilon - 0.75*P**2*UGA_star**2 + 1.0*P**2*UGA_star*epsilon + 0.5*P**2*UGA_star - 0.25*P**2*epsilon - 0.125*P**2 - 0.5*P*R*UGA_star**4*epsilon - 0.25*P*R*UGA_star**4 + 1.0*P*R*UGA_star**3*epsilon + 0.5*P*R*UGA_star**3 + 0.5*P*R*UGA_star**2*epsilon*U_prime(4.0*P*UGA_star**2*epsilon/(4.0*P*UGA_star**2*epsilon + 2.0*P*UGA_star**2 - 8.0*P*UGA_star*epsilon - 4.0*P*UGA_star + 4.0*P*epsilon + 2.0*P + 4.0*R*UGA_star**2*epsilon + 2.0*R*UGA_star**2 - 4.0*S*UGA_star**2*epsilon - 2.0*S*UGA_star**2 + 4.0*S*UGA_star*epsilon + 2.0*S*UGA_star - 4.0*T*UGA_star**2*epsilon - 2.0*T*UGA_star**2 + 4.0*T*UGA_star*epsilon + 2.0*T*UGA_star) - 8.0*P*UGA_star*epsilon/(4.0*P*UGA_star**2*epsilon + 2.0*P*UGA_star**2 - 8.0*P*UGA_star*epsilon - 4.0*P*UGA_star + 4.0*P*epsilon + 2.0*P + 4.0*R*UGA_star**2*epsilon + 2.0*R*UGA_star**2 - 4.0*S*UGA_star**2*epsilon - 2.0*S*UGA_star**2 + 4.0*S*U

Now we can solve the numerator for $U'_{GA}(x^*)$.  This will allow us to write the condition for which the Jacobian will be negative as an upper bound on $U'_{GA}(x^*)$.

In [22]:
upper_bound, = sym.solve(numerator, UGA_prime_star)

ValueError: not enough values to unpack (expected 1, got 0)

In [25]:
sym.factor(upper_bound)

1.0*(2.0*epsilon + 1.0)*(1.0*P*UGA_star**2 - 2.0*P*UGA_star + 1.0*P + 1.0*R*UGA_star**2 - 1.0*S*UGA_star**2 + 1.0*S*UGA_star - 1.0*T*UGA_star**2 + 1.0*T*UGA_star)**2/((2.0*epsilon - 1.0)*(2.0*P*R*UGA_star**2 - 2.0*P*R*UGA_star - 1.0*P*S*UGA_star**2 + 2.0*P*S*UGA_star - 1.0*P*S - 1.0*R*T*UGA_star**2))

After fiddling around with the above expression we arrive at the following condition for a negative Jacobian and a stable interior equilibrium at $0 < x^* < 1$.

\begin{align}
    U'_{GA}(x^*) <& \left(\frac{1 + 2\epsilon}{1 - 2\epsilon}\right)\frac{\bigg(\big((R + P) - (T + S)\big)U_{GA}^{*2} + \big((T + S) - 2P\big)U_{GA}^* + P\bigg)^2}{\big(TR + PS - 2RP\big)U_{GA}^{*2} + 2P(R - S)U_{GA}^* + PS}\\
\end{align}

Can also use a graphical approach to check the correctness of the above condition.  The above condition should be the zero contour in a contour plot of the value of the Jacobian as a function of $U_{GA}$ and $U'_{GA}$.

In [47]:
# vectorized numerical function created from our symbolic expression
_numeric_f_jac = sym.lambdify((UGA_star, UGA_prime_star, T, R, P, S, epsilon), evaluated_f_jac, modules="numpy")

In [48]:
def plot_jacobian(T, R, P, S, epsilon):

    fig, ax = plt.subplots(1, 1, figsize=(20, 10))
    ax.set_ylabel(r"$U_{GA}^*$", fontsize=20, rotation="horizontal")
    ax.set_xlabel(r"$U_{GA}^{'*}$", fontsize=20)
    ax.set_title(r"Negative values of $\frac{\partial \dot{x}_1}{\partial x_1}$ indicate stability!", fontsize=25)
    ax.grid("off")

    equilibrium_selection_probs = np.linspace(0, 1, 100).reshape(-1, 1)
    equilibrium_selection_derivs = np.linspace(0, 10, 1000).reshape(1, -1) # sensible UGA'(0) >= 1
    Z = _numeric_f_jac(equilibrium_selection_probs, equilibrium_selection_derivs, T, R, P, S, epsilon)
    cax = ax.imshow(Z, origin="lower", aspect="auto", vmin=-0.5, vmax=1.0)

    levels = np.arange(-0.25, 1.25, 0.25)
    contours = ax.contour(Z, levels=levels, colors='w', origin='lower')
    ax.clabel(contours, contours.levels, inline=True, fontsize=10)
    
    # adjust the tick labels
    locs, _ = plt.xticks()
    plt.xticks(locs[1:], np.linspace(0, 10, locs.size-1))
    locs, _ = plt.yticks()
    plt.yticks(locs[1:], np.linspace(0, 1, locs.size-1))

    plt.show()

In [49]:
mpld3.disable_notebook()  # don't need interactive plotting for this!

# sliders used to control the Prisoner's Dilemma Payoffs
T_slider = widgets.FloatSlider(value=25, min=0, max=100, step=0.1, description=r"$T$")
R_slider = widgets.FloatSlider(value=3, min=0, max=100, step=0.1, description=r"$R$")
P_slider = widgets.FloatSlider(value=2, min=0, max=100, step=0.1, description=r"$P$")
S_slider = widgets.FloatSlider(value=1, min=0, max=100, step=0.1, description=r"$S$")

e_slider = widgets.FloatSlider(value=0, min=0, max=1, step=1e-3, description=r"$\epsilon$", readout_format=".3f")

w = widgets.interactive(plot_jacobian, T=T_slider, R=R_slider, P=P_slider, S=S_slider, epsilon=e_slider)
display(w)

A Jupyter Widget

# Invadability

Restrict the full model to obtain a monomorphic $\gamma$ equilibrium, add a small amount of individuals carrying the $g$ allele of the $\gamma$ gene and simulate.

In [26]:
x2, x3 = sym.symbols("x2, x3", real=True, positive=True)
x4 = 1 - x1 - x2 - x3
x = np.array([[x1], [x2], [x3], [x4]])
W = models.generalized_sexual_selection(x, UGA, UgA, payoff_kernel, mutation_rate=epsilon)
f1, f2, f3, _ = models.offspring_genotypes_evolution(W, x)

In [27]:
F = sym.Matrix([f1, f2, f3])

In [28]:
F_jac = F.jacobian([x1, x2, x3])

In [29]:
UgA_star = sym.symbols("UgA_star", positive=True, real=True)

evaluated_F_jac = (F_jac.subs({x2: 1 - x1, x3: 0})
                        .subs({UGA(x1): UGA_star, UgA(x1): UgA_star})
                        .doit()
                        .subs({sym.Derivative(UGA(x1), x1): UGA_prime_star}))

In [31]:
simplified_F_jac = sym.zeros(3, 3)

for i in range(3):
    for j in range(3):
        simplified_F_jac[i, j] = sym.factor(sym.cancel(sym.together(evaluated_F_jac[i, j])))
        print("Finished with element {},{}!".format(i,j))


Finished with element 0,0!
Finished with element 0,1!
Finished with element 0,2!
Finished with element 1,0!
Finished with element 1,1!
Finished with element 1,2!
Finished with element 2,0!
Finished with element 2,1!
Finished with element 2,2!


In [94]:
charpoly = simplified_F_jac.charpoly()

In [36]:
eigenvals = simplified_F_jac.eigenvals()

In [114]:
import pickle

with open('non-invadable-eigenvalues.pickle', 'wb') as handle:
    pickle.dump(eigenvals, handle, protocol=pickle.HIGHEST_PROTOCOL)

NameError: name 'eigenvals' is not defined

In [38]:
e1, e2, e3 = eigenvals.keys()

In [42]:
numerator, denominator = sym.fraction(sym.factor(e1))

In [46]:
denominator

2*(P*UGA_star**2 - 2*P*UGA_star + P + R*UGA_star**2 - S*UGA_star**2 + S*UGA_star - T*UGA_star**2 + T*UGA_star)**2

In [48]:
upper_bound, = sym.solve(numerator, UGA_prime_star)

In [50]:
sym.factor(upper_bound)

(2*epsilon + 1)*(P*UGA_star**2 - 2*P*UGA_star + P + R*UGA_star**2 - S*UGA_star**2 + S*UGA_star - T*UGA_star**2 + T*UGA_star)**2/((2*epsilon - 1)*(2*P*R*UGA_star**2 - 2*P*R*UGA_star - P*S*UGA_star**2 + 2*P*S*UGA_star - P*S - R*T*UGA_star**2))

In [95]:
e1.subs({UGA_star: 1, UGA_prime_star: 0})

-(2*R**2*epsilon + R**2)/(2*R**2)

In [73]:
# vectorized numeric repr for the eigenvalue
_numeric_e2 = sym.lambdify((UGA_star, UgA_star, T, R, P, S, epsilon),
                           e2.subs({x1: x1_star}),
                           modules="numpy")

In [75]:
def plot_second_eigenvalue(T, R, P, S, epsilon):

    fig, ax = plt.subplots(1, 1, figsize=(10, 10))
    ax.set_ylabel(r"$U_{GA}^*$", fontsize=20, rotation="horizontal")
    ax.set_xlabel(r"$U_{gA}^{*}$", fontsize=20)
    ax.set_title(r"Eigenvalue, $e_2$", fontsize=25)
    ax.grid("off")

    equilibrium_selection_probs = np.linspace(0, 1, 100)
    UgAs, UGAs = np.meshgrid(equilibrium_selection_probs, equilibrium_selection_probs)
    Z = _numeric_e2(UGAs, UgAs, T, R, P, S, epsilon)
    cax = ax.imshow(Z, origin="lower")
    contours = ax.contour(Z, colors='w', origin='lower')
    ax.clabel(contours, contours.levels, inline=True, fontsize=10)
    
    # adjust the tick labels
    locs, _ = plt.xticks()
    plt.xticks(locs[1:], np.linspace(0, 1, locs.size-1))
    locs, _ = plt.yticks()
    plt.yticks(locs[1:], np.linspace(0, 1, locs.size-1))

    plt.show()

In [80]:
mpld3.disable_notebook()  # don't need interactive plotting for this!

# sliders used to control the Prisoner's Dilemma Payoffs
T_slider = widgets.FloatSlider(value=10, min=0, max=100, step=0.1, description=r"$T$")
R_slider = widgets.FloatSlider(value=8, min=0, max=100, step=0.1, description=r"$R$")
P_slider = widgets.FloatSlider(value=6, min=0, max=100, step=0.1, description=r"$P$")
S_slider = widgets.FloatSlider(value=4, min=0, max=100, step=0.1, description=r"$S$")

# slider used to control the mutation rate
e_slider = widgets.FloatSlider(value=0.0, min=0.0, max=1.0, step=1e-3, description=r"$\epsilon$", readout_format=".3f")

w = widgets.interactive(plot_second_eigenvalue, T=T_slider, R=R_slider, P=P_slider, S=S_slider, epsilon=e_slider)
display(w)

A Jupyter Widget

In [105]:
evaluated = (e2.subs({x1: x1_star, UgA_star: x1_star})
               .subs({UGA_star: 1, UGA_prime_star: 0}))
together = sym.together(evaluated)

In [107]:
expanded = sym.expand(together)

In [108]:
expanded

-4.0*P*epsilon**4/(16.0*R*epsilon**3 + 16.0*R*epsilon**2 + 4.0*R*epsilon) + 6.0*P*epsilon**3/(16.0*R*epsilon**3 + 16.0*R*epsilon**2 + 4.0*R*epsilon) - 12.0*R*epsilon**3/(16.0*R*epsilon**3 + 16.0*R*epsilon**2 + 4.0*R*epsilon) - 13.0*R*epsilon**2/(16.0*R*epsilon**3 + 16.0*R*epsilon**2 + 4.0*R*epsilon) - 1.5*R*epsilon/(16.0*R*epsilon**3 + 16.0*R*epsilon**2 + 4.0*R*epsilon) - 2.0*S*epsilon**3/(16.0*R*epsilon**3 + 16.0*R*epsilon**2 + 4.0*R*epsilon) + 3.0*S*epsilon**2/(16.0*R*epsilon**3 + 16.0*R*epsilon**2 + 4.0*R*epsilon) - 2.0*T*epsilon**3/(16.0*R*epsilon**3 + 16.0*R*epsilon**2 + 4.0*R*epsilon) + 3.0*T*epsilon**2/(16.0*R*epsilon**3 + 16.0*R*epsilon**2 + 4.0*R*epsilon) + 0.015625*sqrt(65536.0*P**2*epsilon**8 + 65536.0*P**2*epsilon**7 + 16384.0*P**2*epsilon**6 + 524288.0*P*R*epsilon**8 + 655360.0*P*R*epsilon**7 + 229376.0*P*R*epsilon**6 - 8192.0*P*R*epsilon**4 + 65536.0*P*S*epsilon**7 + 65536.0*P*S*epsilon**6 + 16384.0*P*S*epsilon**5 + 65536.0*P*T*epsilon**7 + 65536.0*P*T*epsilon**6 + 16384.

In [110]:
tmp_expr = sym.together(expanded)

In [111]:
tmp_expr

(-4.0*P*epsilon**3 + 6.0*P*epsilon**2 - 12.0*R*epsilon**2 - 13.0*R*epsilon - 1.5*R - 2.0*S*epsilon**2 + 3.0*S*epsilon - 2.0*T*epsilon**2 + 3.0*T*epsilon + 0.015625*sqrt(65536.0*P**2*epsilon**6 + 65536.0*P**2*epsilon**5 + 16384.0*P**2*epsilon**4 + 524288.0*P*R*epsilon**6 + 655360.0*P*R*epsilon**5 + 229376.0*P*R*epsilon**4 - 8192.0*P*R*epsilon**2 + 65536.0*P*S*epsilon**5 + 65536.0*P*S*epsilon**4 + 16384.0*P*S*epsilon**3 + 65536.0*P*T*epsilon**5 + 65536.0*P*T*epsilon**4 + 16384.0*P*T*epsilon**3 + 65536.0*R**2*epsilon**4 + 163840.0*R**2*epsilon**3 + 151552.0*R**2*epsilon**2 + 61440.0*R**2*epsilon + 9216.0*R**2 + 65536.0*R*S*epsilon**4 + 114688.0*R*S*epsilon**3 + 65536.0*R*S*epsilon**2 + 12288.0*R*S*epsilon + 262144.0*R*T*epsilon**5 + 327680.0*R*T*epsilon**4 + 114688.0*R*T*epsilon**3 - 4096.0*R*T*epsilon + 16384.0*S**2*epsilon**4 + 16384.0*S**2*epsilon**3 + 4096.0*S**2*epsilon**2 + 32768.0*S*T*epsilon**4 + 32768.0*S*T*epsilon**3 + 8192.0*S*T*epsilon**2 + 16384.0*T**2*epsilon**4 + 16384.0*T*

In [117]:
tmp_expr.subs({epsilon**2: 0})

(-13.0*R*epsilon - 1.5*R + 3.0*S*epsilon + 3.0*T*epsilon + 0.015625*sqrt(61440.0*R**2*epsilon + 9216.0*R**2 + 12288.0*R*S*epsilon - 4096.0*R*T*epsilon))/(R*(16.0*epsilon + 4.0))

\begin{align}
   % (-13.0*R*epsilon - 1.5*R + 3.0*S*epsilon + 3.0*T*epsilon + 0.015625*sqrt(61440.0*R**2*epsilon + 9216.0*R**2 + 12288.0*R*S*epsilon - 4096.0*R*T*epsilon))/(R*(16.0*epsilon + 4.0)) < 0 \\
    \frac{(-13R\epsilon - 1.5R + 3S\epsilon + 3T\epsilon + 0.015625\sqrt{61440R^2\epsilon + 9216R^2 + 12288RS\epsilon - 4096RT\epsilon}}{R(16\epsilon + 4.0)} < 0 \\
    \frac{(-13R\epsilon - \frac{3}{2}R + 3S\epsilon + 3T\epsilon + \frac{1}{64}\sqrt{61440R^2\epsilon + 9216R^2 + 12288RS\epsilon - 4096RT\epsilon}}{R(16\epsilon + 4.0)} < 0 \\
    \frac{(-13R\epsilon - \frac{3}{2}R + 3S\epsilon + 3T\epsilon + \frac{1}{2}\sqrt{60R^2\epsilon + 9R^2 + 12RS\epsilon - 4RT\epsilon}}{R(16\epsilon + 4)} < 0
\end{align}

In [177]:
sym.together(sym.expand(sym.together(e2.subs({x1: x1_star, UgA_star: x1_star})))).subs({epsilon**2: 0})

(192.0*P**5*UGA_star**10*epsilon - 1920.0*P**5*UGA_star**9*epsilon + 8544.0*P**5*UGA_star**8*epsilon - 22272.0*P**5*UGA_star**7*epsilon + 37632.0*P**5*UGA_star**6*epsilon - 43008.0*P**5*UGA_star**5*epsilon + 33600.0*P**5*UGA_star**4*epsilon - 17664.0*P**5*UGA_star**3*epsilon + 5952.0*P**5*UGA_star**2*epsilon - 1152.0*P**5*UGA_star*epsilon + 96.0*P**5*epsilon + 864.0*P**4*R*UGA_star**10*epsilon + 80.0*P**4*R*UGA_star**10 - 6912.0*P**4*R*UGA_star**9*epsilon - 640.0*P**4*R*UGA_star**9 + 24224.0*P**4*R*UGA_star**8*epsilon + 2192.0*P**4*R*UGA_star**8 - 48576.0*P**4*R*UGA_star**7*epsilon - 4192.0*P**4*R*UGA_star**7 + 60960.0*P**4*R*UGA_star**6*epsilon + 4880.0*P**4*R*UGA_star**6 - 49024.0*P**4*R*UGA_star**5*epsilon - 3520.0*P**4*R*UGA_star**5 + 24672.0*P**4*R*UGA_star**4*epsilon + 1520.0*P**4*R*UGA_star**4 - 7104.0*P**4*R*UGA_star**3*epsilon - 352.0*P**4*R*UGA_star**3 + 896.0*P**4*R*UGA_star**2*epsilon + 32.0*P**4*R*UGA_star**2 - 864.0*P**4*S*UGA_star**10*epsilon - 80.0*P**4*S*UGA_star**10 +

In [60]:
numerator, denominator = sym.fraction(sym.factor(e2))

In [61]:
denominator

8*x1*(x1 - 1)*(P*UGA_star**2 - 2*P*UGA_star + P + R*UGA_star**2 - S*UGA_star**2 + S*UGA_star - T*UGA_star**2 + T*UGA_star)

In [77]:
# vectorized numeric repr for the eigenvalue
_numeric_e3 = sym.lambdify((UGA_star, UgA_star, T, R, P, S, epsilon),
                           e3.subs({x1: x1_star}),
                           modules="numpy")

In [78]:
def plot_third_eigenvalue(T, R, P, S, epsilon):

    fig, ax = plt.subplots(1, 1, figsize=(10, 10))
    ax.set_ylabel(r"$U_{GA}^*$", fontsize=20, rotation="horizontal")
    ax.set_xlabel(r"$U_{gA}^{*}$", fontsize=20)
    ax.set_title(r"Eigenvalue, $e_3$", fontsize=25)
    ax.grid("off")

    equilibrium_selection_probs = np.linspace(0, 1, 100)
    UgAs, UGAs = np.meshgrid(equilibrium_selection_probs, equilibrium_selection_probs)
    Z = _numeric_e3(UGAs, UgAs, T, R, P, S, epsilon)
    cax = ax.imshow(Z, origin="lower")
    contours = ax.contour(Z, colors='w', origin='lower')
    ax.clabel(contours, contours.levels, inline=True, fontsize=10)
    
    # adjust the tick labels
    locs, _ = plt.xticks()
    plt.xticks(locs[1:], np.linspace(0, 1, locs.size-1))
    locs, _ = plt.yticks()
    plt.yticks(locs[1:], np.linspace(0, 1, locs.size-1))

    plt.show()

In [79]:
mpld3.disable_notebook()  # don't need interactive plotting for this!

# sliders used to control the Prisoner's Dilemma Payoffs
T_slider = widgets.FloatSlider(value=25, min=0, max=100, step=0.1, description=r"$T$")
R_slider = widgets.FloatSlider(value=3, min=0, max=100, step=0.1, description=r"$R$")
P_slider = widgets.FloatSlider(value=2, min=0, max=100, step=0.1, description=r"$P$")
S_slider = widgets.FloatSlider(value=1, min=0, max=100, step=0.1, description=r"$S$")

# slider used to control the mutation rate
e_slider = widgets.FloatSlider(value=0.0, min=0.0, max=1.0, step=1e-3, description=r"$\epsilon$", readout_format=".3f")

w = widgets.interactive(plot_third_eigenvalue, T=T_slider, R=R_slider, P=P_slider, S=S_slider, epsilon=e_slider)
display(w)

A Jupyter Widget

In [158]:
e2 == e3

False

In [164]:
sym.simplify(e2.subs({x1: x1_star, UgA_star: x1_star}).subs({UGA_star: 1, UGA_prime_star: 0})).subs({epsilon**2: 0})

(-13.0*R*epsilon - 1.5*R + 3.0*S*epsilon + 3.0*T*epsilon + 0.015625*sqrt(61440.0*R**2*epsilon + 9216.0*R**2 + 12288.0*R*S*epsilon - 4096.0*R*T*epsilon))/(R*(16.0*epsilon + 4.0))

In [165]:
sym.simplify(e3.subs({x1: x1_star, UgA_star: x1_star}).subs({UGA_star: 1, UGA_prime_star: 0})).subs({epsilon**2: 0})

(-13.0*R*epsilon - 1.5*R + 3.0*S*epsilon + 3.0*T*epsilon - 0.015625*sqrt(61440.0*R**2*epsilon + 9216.0*R**2 + 12288.0*R*S*epsilon - 4096.0*R*T*epsilon))/(R*(16.0*epsilon + 4.0))

In [153]:
evaluated = (e3.subs({x1: x1_star, UgA_star: x1_star})
               .subs({UGA_star: 1, UGA_prime_star: 0}))
together = sym.together(evaluated)

In [154]:
expanded = sym.expand(together)

In [ ]:
tmp_expr = sym.together(expanded)

In [155]:
tmp_expr.subs({epsilon**2: 0})

(-13.0*R*epsilon - 1.5*R + 3.0*S*epsilon + 3.0*T*epsilon + 0.015625*sqrt(61440.0*R**2*epsilon + 9216.0*R**2 + 12288.0*R*S*epsilon - 4096.0*R*T*epsilon))/(R*(16.0*epsilon + 4.0))

In [ ]:
(-13.0*R*epsilon - 1.5*R + 3.0*S*epsilon + 3.0*T*epsilon + 0.015625*sqrt(61440.0*R**2*epsilon + 9216.0*R**2 + 12288.0*R*S*epsilon - 4096.0*R*T*epsilon))/(R*(16.0*epsilon + 4.0))

First eigenvalue can be written as a hyperbola in $U_{gA}^*,U_{GA}^*$ plane.

$$ e_1 = \left(\frac{\big((R + P) - (T + S)\big)U_{gA}^{*2} + \big((T + S) - 2P\big)U_{gA}^*}{N^*}\right) - \left(\frac{\big((R + P) - (T + S)\big)U_{GA}^{*2} + \big((T + S) - 2P\big)U_{GA}^*}{N^*}\right) $$

### Completing the square

To write this hyperbola in standard form we need to [complete the square](https://en.wikipedia.org/wiki/Completing_the_square). Completing the square for the quadratic polynomial in $U_{GA}(x^*)$ yields the following.

\begin{align}
    \big((R + P) - (T + S)\big)\left(U_{GA}^* - \bar{U}_{GA}^*\right)^2 - \frac{1}{2}\big(\bar{N}^* - 2P\big) \\
\end{align}

where $\bar{U}_{GA}^*$ is the value of $U_{GA}^*$ that maximizes total offspring $N$ derived above. Completing the square for the quadratic polynomial in $U_{gA}(x^*)$ yields a similar expression.

\begin{align}
    \big((R + P) - (T + S)\big)\left(U_{gA}^* - \bar{U}_{GA}^*\right)^2 - \frac{1}{2}\big(\bar{N}^* - 2P\big) \\
\end{align}

Substituting these results into the expression for the eigenvalue yields the following.

$$ e_1 = \left(\frac{\big(U_{gA}^* - \bar{U}_{GA}^*\big)^{2}}{\frac{N^*}{\big((R + P) - (T + S)\big)}}\right) - \left(\frac{\big(U_{GA}^* - \bar{U}_{GA}^*\big)^{2}}{\frac{N^*}{\big((R + P) - (T + S)\big)}}\right) $$


### Conditions for negative eigenvalue

Non-invadability requires that this eigenvalue is strictly negative.

$$ \left(\frac{\big(U_{gA}^* - \bar{U}_{GA}^*\big)^{2}}{\frac{N^*}{\big((R + P) - (T + S)\big)}}\right) - \left(\frac{\big(U_{GA}^* - \bar{U}_{GA}^*\big)^{2}}{\frac{N^*}{\big((R + P) - (T + S)\big)}}\right) < 0$$

#### Case: (R + P) < (T + S)
In this case the inequality simplifies to the following.

$$ \big(U_{gA}^* - \bar{U}_{GA}^*\big)^{2} - \big(U_{GA}^* - \bar{U}_{GA}^*\big)^{2} > 0 $$

This implies that we have four sub-cases to consider.

\begin{align}
    \big(U_{gA}^* - \bar{U}_{GA}^*\big) >& \big(U_{GA}^* - \bar{U}_{GA}^*\big) \\
    \big(U_{gA}^* - \bar{U}_{GA}^*\big) >& -\big(U_{GA}^* - \bar{U}_{GA}^*\big) \\
    -\big(U_{gA}^* - \bar{U}_{GA}^*\big) >& \big(U_{GA}^* - \bar{U}_{GA}^*\big) \\
    -\big(U_{gA}^* - \bar{U}_{GA}^*\big) >& -\big(U_{GA}^* - \bar{U}_{GA}^*\big)
\end{align}

These four sub-cases define the following two regions of the $U_{gA}^*,U_{GA}^*$ plane where the eigenvalue is negative.

\begin{align}
   & 2\big(\bar{U}_{GA}^* - U_{gA}^*\big) < U_{GA}^* - U_{gA}^* < 0\\
   & 0 < U_{GA}^* - U_{gA}^* < 2\big(\bar{U}_{GA}^* - U_{gA}\big)
\end{align}

#### Case: (R + P) > (T + S)
In this case the inequality simplifies to the following.

$$ \big(U_{gA}^* - \bar{U}_{GA}^*\big)^{2} - \big(U_{GA}^* - \bar{U}_{GA}^*\big)^{2} < 0 $$

This implies that we again have four sub-cases to consider.

\begin{align}
    \big(U_{gA}^* - \bar{U}_{GA}^*\big) <& \big(U_{GA}^* - \bar{U}_{GA}^*\big) \\
    \big(U_{gA}^* - \bar{U}_{GA}^*\big) <& -\big(U_{GA}^* - \bar{U}_{GA}^*\big) \\
    -\big(U_{gA}^* - \bar{U}_{GA}^*\big) <& \big(U_{GA}^* - \bar{U}_{GA}^*\big) \\
    -\big(U_{gA}^* - \bar{U}_{GA}^*\big) <& -\big(U_{GA}^* - \bar{U}_{GA}^*\big)
\end{align}

These four sub-cases define the following two regions of the $U_{gA}^*,U_{GA}^*$ plane where the eigenvalue is negative.

\begin{align}
   & 2\big(\bar{U}_{GA}^* - U_{gA}^*\big) < U_{GA}^* - U_{gA}^* < 0\\
   & 0 < U_{GA}^* - U_{gA}^* < 2\big(\bar{U}_{GA}^* - U_{gA}\big)
\end{align}


### Asymptotes 

Asymptotes of this hyperbola can be defined as follows.

$$ U_{GA}^* - \bar{U}_{GA}^* = \pm\big(U_{gA}^* - \bar{U}_{GA}^*\big) $$

The above can be written as linear equations in $U_{gA}^*,U_{GA}^*$ plane.

\begin{align}
    U_{GA}^* =& U_{gA}^* \\
    U_{GA}^* =& 2\bar{U}_{GA}^* - U_{gA}^*
\end{align}

Note that the asymptotes of this hyperbola correspond to the locus of points for which the value of the first eigenvalue is exactly zero.

#### Discussion

Suppose $U_{GA}^* = \bar{U}_{GA}^*$, then both of the sets of inequalities will hold for any value of $U_{gA}^*$. This implies that the eigenvalue will be negative for any value of $U_{gA}^*$.

In [83]:
# vectorized numeric repr for the eigenvalue
_numeric_e1 = sym.lambdify((UGA_star, UGA_prime_star, T, R, P, S, M), e1, modules="numpy")

In [84]:
def plot_first_eigenvalue(T, R, P, S, M):

    fig, ax = plt.subplots(1, 1, figsize=(10, 10))
    ax.set_ylabel(r"$U_{GA}^*$", fontsize=20, rotation="horizontal")
    ax.set_xlabel(r"$U_{GA}^{'*}$", fontsize=20)
    ax.set_title(r"Eigenvalue, $e_1$", fontsize=25)
    ax.grid("off")

    equilibrium_selection_probs = np.linspace(0, 1, 100).reshape(-1, 1)
    equilibrium_selection_derivs = np.linspace(0, 10, 1000).reshape(1, -1) # sensible UGA'(0) >= 1
    Z = _numeric_e1(equilibrium_selection_probs, equilibrium_selection_derivs, T, R, P, S, M)
    cax = ax.imshow(Z, origin="lower", aspect="auto", vmin=-0.5, vmax=1.0)

    levels = np.arange(-0.25, 1.25, 0.25)
    contours = ax.contour(Z, levels=levels, colors='w', origin='lower')
    ax.clabel(contours, contours.levels, inline=True, fontsize=10)
    
    # adjust the tick labels
    locs, _ = plt.xticks()
    plt.xticks(locs[1:], np.linspace(0, 10, locs.size-1))
    locs, _ = plt.yticks()
    plt.yticks(locs[1:], np.linspace(0, 1, locs.size-1))

    plt.show()

In [85]:
mpld3.disable_notebook()  # don't need interactive plotting for this!

# sliders used to control the Prisoner's Dilemma Payoffs
T_slider = widgets.FloatSlider(value=25, min=0, max=100, step=0.1, description=r"$T$")
R_slider = widgets.FloatSlider(value=3, min=0, max=100, step=0.1, description=r"$R$")
P_slider = widgets.FloatSlider(value=2, min=0, max=100, step=0.1, description=r"$P$")
S_slider = widgets.FloatSlider(value=1, min=0, max=100, step=0.1, description=r"$S$")

M_slider = widgets.FloatSlider(value=1, min=0, max=100, step=0.1, description=r"$M$")

w = widgets.interactive(plot_first_eigenvalue, T=T_slider, R=R_slider, P=P_slider, S=S_slider, M=M_slider)
display(w)

A Jupyter Widget

In [51]:
UGA_prime_star, = sym.solve(numerator, UGA_prime_star)

In [52]:
sym.factor(UGA_prime_star)

-(P*UGA_star**2 - 2*P*UGA_star + P + R*UGA_star**2 - S*UGA_star**2 + S*UGA_star - T*UGA_star**2 + T*UGA_star)**2/(2*P*R*UGA_star**2 - 2*P*R*UGA_star - P*S*UGA_star**2 + 2*P*S*UGA_star - P*S - R*T*UGA_star**2)

After a bit of fiddling, we arrive at the same stability condition derived above.

\begin{align}
   % U'_{GA}(x^*) < -\frac{(PU_{GA}(x^*)^2 - 2PU_{GA}(x^*) + P + RU_{GA}(x^*)^2 - SU_{GA}(x^*)^2 + SU_{GA}(x^*) - TU_{GA}(x^*)^2 + TU_{GA}(x^*))^2}{(2PRU_{GA}(x^*)^2 - 2PRU_{GA}(x^*) - PSU_{GA}(x^*)^2 + 2PSU_{GA}(x^*) - PS - RTU_{GA}(x^*)^2)} \\
    U'_{GA}(x^*) < \frac{\bigg(\big((R + P) - (T + S)\big)U_{GA}(x^*)^2 + \big((T + S) - 2P\big)U_{GA}(x^*) + P\bigg)^2}{\big(R(T-2P) + PS\big)U_{GA}(x^*)^2 + 2P(R - S)U_{GA}(x^*) + PS}
\end{align}

In [72]:
e2

(-3*M + 3*P*UGA_star**2 - 6*P*UGA_star - P*UgA_star**2 + 2*P*UgA_star + 2*P + 3*R*UGA_star**2 - R*UgA_star**2 - 3*S*UGA_star**2 + 3*S*UGA_star + S*UgA_star**2 - S*UgA_star - 3*T*UGA_star**2 + 3*T*UGA_star + T*UgA_star**2 - T*UgA_star + m)/(4*M - 4*P*UGA_star**2 + 8*P*UGA_star - 4*P - 4*R*UGA_star**2 + 4*S*UGA_star**2 - 4*S*UGA_star + 4*T*UGA_star**2 - 4*T*UGA_star)

$$ e_2 = \left(\frac{\big((R + P) - (T + S)\big)U_{gA}^{*2} + \big((T + S) - 2P\big)U_{gA}^* + P}{4N^*}\right) - \frac{3}{4} $$

### Completing the square

Completing the square for the quadratic polynomial in $U_{gA}(x^*)$ yields the following expression.

\begin{align}
    \big((R + P) - (T + S)\big)\left(U_{gA}^* - \bar{U}_{GA}^*\right)^2 + \frac{1}{2}\bar{N}^* \\
\end{align}

Finally we need to find conditions under which the third eigenvalue is strictly negative.

In [78]:
# vectorized numeric repr for the eigenvalue
_numeric_e2 = sym.lambdify((UGA_star, UgA_star, T, R, P, S, M, m), e2, modules="numpy")

In [79]:
def plot_second_eigenvalue(T, R, P, S, M, m):

    fig, ax = plt.subplots(1, 1, figsize=(10, 10))
    ax.set_ylabel(r"$U_{GA}^*$", fontsize=20, rotation="horizontal")
    ax.set_xlabel(r"$U_{gA}^{*}$", fontsize=20)
    ax.set_title(r"Eigenvalue, $e_2$", fontsize=25)
    ax.grid("off")

    equilibrium_selection_probs = np.linspace(0, 1, 100)
    UgAs, UGAs = np.meshgrid(equilibrium_selection_probs, equilibrium_selection_probs)
    Z = _numeric_e2(UGAs, UgAs, T, R, P, S, M, m)
    cax = ax.imshow(Z, origin="lower")
    contours = ax.contour(Z, colors='w', origin='lower')
    ax.clabel(contours, contours.levels, inline=True, fontsize=10)
    
    # adjust the tick labels
    locs, _ = plt.xticks()
    plt.xticks(locs[1:], np.linspace(0, 1, locs.size-1))
    locs, _ = plt.yticks()
    plt.yticks(locs[1:], np.linspace(0, 1, locs.size-1))

    plt.show()

In [80]:
mpld3.disable_notebook()  # don't need interactive plotting for this!

# sliders used to control the Prisoner's Dilemma Payoffs
T_slider = widgets.FloatSlider(value=25, min=0, max=100, step=0.1, description=r"$T$")
R_slider = widgets.FloatSlider(value=3, min=0, max=100, step=0.1, description=r"$R$")
P_slider = widgets.FloatSlider(value=2, min=0, max=100, step=0.1, description=r"$P$")
S_slider = widgets.FloatSlider(value=1, min=0, max=100, step=0.1, description=r"$S$")

M_slider = widgets.FloatSlider(value=1, min=0, max=100, step=0.1, description=r"$M$")
m_slider = widgets.FloatSlider(value=0, min=0, max=100, step=0.1, description=r"$m$")

w = widgets.interactive(plot_second_eigenvalue, T=T_slider, R=R_slider, P=P_slider, S=S_slider, M=M_slider, m=m_slider)
display(w)

A Jupyter Widget

In [81]:
e3

(-M + P*UGA_star**2 - 2*P*UGA_star - P*UgA_star**2 + 2*P*UgA_star + R*UGA_star**2 - R*UgA_star**2 - S*UGA_star**2 + S*UGA_star + S*UgA_star**2 - S*UgA_star - T*UGA_star**2 + T*UGA_star + T*UgA_star**2 - T*UgA_star + m)/(2*M - 2*P*UGA_star**2 + 4*P*UGA_star - 2*P - 2*R*UGA_star**2 + 2*S*UGA_star**2 - 2*S*UGA_star + 2*T*UGA_star**2 - 2*T*UGA_star)

In [86]:
# vectorized numeric repr for the eigenvalue
_numeric_e3 = sym.lambdify((UGA_star, UgA_star, T, R, P, S, M, m), e3, modules="numpy")

In [89]:
def plot_third_eigenvalue(T, R, P, S, M, m):

    fig, ax = plt.subplots(1, 1, figsize=(10, 10))
    ax.set_ylabel(r"$U_{GA}^*$", fontsize=20, rotation="horizontal")
    ax.set_xlabel(r"$U_{gA}^{*}$", fontsize=20)
    ax.set_title(r"Eigenvalue, $e_3$", fontsize=25)
    ax.grid("off")

    equilibrium_selection_probs = np.linspace(0, 1, 100)
    UgAs, UGAs = np.meshgrid(equilibrium_selection_probs, equilibrium_selection_probs)
    Z = _numeric_e3(UGAs, UgAs, T, R, P, S, M, m)
    cax = ax.imshow(Z, origin="lower")
    contours = ax.contour(Z, colors='w', origin='lower')
    ax.clabel(contours, contours.levels, inline=True, fontsize=10)
    
    # adjust the tick labels
    locs, _ = plt.xticks()
    plt.xticks(locs[1:], np.linspace(0, 1, locs.size-1))
    locs, _ = plt.yticks()
    plt.yticks(locs[1:], np.linspace(0, 1, locs.size-1))

    plt.show()

In [90]:
mpld3.disable_notebook()  # don't need interactive plotting for this!

# sliders used to control the Prisoner's Dilemma Payoffs
T_slider = widgets.FloatSlider(value=25, min=0, max=100, step=0.1, description=r"$T$")
R_slider = widgets.FloatSlider(value=3, min=0, max=100, step=0.1, description=r"$R$")
P_slider = widgets.FloatSlider(value=2, min=0, max=100, step=0.1, description=r"$P$")
S_slider = widgets.FloatSlider(value=1, min=0, max=100, step=0.1, description=r"$S$")

M_slider = widgets.FloatSlider(value=1, min=0, max=100, step=0.1, description=r"$M$")
m_slider = widgets.FloatSlider(value=0, min=0, max=100, step=0.1, description=r"$m$")

w = widgets.interactive(plot_third_eigenvalue, T=T_slider, R=R_slider, P=P_slider, S=S_slider, M=M_slider, m=m_slider)
display(w)

A Jupyter Widget

## When is the fitness-maximizing interior equilibrium un-invadable by randomista?

In [147]:
evaluated_e3 = e3.subs({UGA_star: optimal_UGA_star, UgA_star: optimal_x1_star, m: 0})
simplified_e3 = sym.factor(sym.cancel(sym.together(evaluated_e3)))

In [148]:
numerator, denominator = sym.fraction(simplified_e3)

In [151]:
numerator

-4.0*M**3*P**3 - 12.0*M**3*P**2*R + 12.0*M**3*P**2*S + 12.0*M**3*P**2*T - 12.0*M**3*P*R**2 + 24.0*M**3*P*R*S + 24.0*M**3*P*R*T - 12.0*M**3*P*S**2 - 24.0*M**3*P*S*T - 12.0*M**3*P*T**2 - 4.0*M**3*R**3 + 12.0*M**3*R**2*S + 12.0*M**3*R**2*T - 12.0*M**3*R*S**2 - 24.0*M**3*R*S*T - 12.0*M**3*R*T**2 + 4.0*M**3*S**3 + 12.0*M**3*S**2*T + 12.0*M**3*S*T**2 + 4.0*M**3*T**3 + 8.0*M**2*P**3*R + 16.0*M**2*P**2*R**2 - 16.0*M**2*P**2*R*S - 16.0*M**2*P**2*R*T - 2.0*M**2*P**2*S**2 - 4.0*M**2*P**2*S*T - 2.0*M**2*P**2*T**2 + 8.0*M**2*P*R**3 - 16.0*M**2*P*R**2*S - 16.0*M**2*P*R**2*T + 4.0*M**2*P*R*S**2 + 8.0*M**2*P*R*S*T + 4.0*M**2*P*R*T**2 + 4.0*M**2*P*S**3 + 12.0*M**2*P*S**2*T + 12.0*M**2*P*S*T**2 + 4.0*M**2*P*T**3 - 2.0*M**2*R**2*S**2 - 4.0*M**2*R**2*S*T - 2.0*M**2*R**2*T**2 + 4.0*M**2*R*S**3 + 12.0*M**2*R*S**2*T + 12.0*M**2*R*S*T**2 + 4.0*M**2*R*T**3 - 2.0*M**2*S**4 - 8.0*M**2*S**3*T - 12.0*M**2*S**2*T**2 - 8.0*M**2*S*T**3 - 2.0*M**2*T**4 - 4.0*M*P**3*R**2 - 4.0*M*P**2*R**3 + 4.0*M*P**2*R**2*S + 4.0*M*P*

\begin{align}
    4\bigg(-\big(P^3 + 3P^2R + 3PR^2 + R^3\big) - \big(P^3 - 3P^2S + 3PS^2 + S^3\big) + \big(-P^3 + 3P^2T - 3PT^2 + T^3\big) - \big(R^3 - 3R^2S + 3RS^2 - S^3\big) + \big(T^3 - 3RT^2 + 3R^2T - R^3\big) + 2P^3 + 2R^3 + 8PRS + 8PRT - 8PST - 8RST - 2T^3 + \big(S^3 + 3S^2T + 3ST^2 + T^3\big)\bigg)M^3 + 2\big(4P^3R + 8P^2R^2 - 8P^2RS - 8P^2RT - P^2S^2 - 2P^2ST - P^2T^2 + 4PR^3 - 8PR^2S - 8PR^2T + 2PRS^2 + 4PRST + 2PRT^2 + 2PS^3 + 6PS^2T + 6PST^2 + 2PT^3 - R^2S^2 - 2R^2ST - R^2T^2 + 2RS^3 + 6RS^2T + 6RST^2 + 2RT^3 - S^4 - 4S^3T - 6S^2T^2 - 4ST^3 - T^4\big)M^2 + \big(- 4P^3R^2 - 4P^2R^3 + 4P^2R^2S + 4P^2R^2T + 2P^2RS^2 + 4P^2RST + 2P^2RT^2 + 2PR^2S^2 + 4PR^2ST + 2PR^2T^2 - 2PRS^3 - 6PRS^2T - 6PRST^2 - 2PRT^3 - 0.25PS^4 -PS^3T - 1.5PS^2T^2 - PST^3 - 0.25PT^4 - 0.25RS^4 - RS^3T - 1.5RS^2T^2 - RST^3 - 0.25RT^4 + 0.25S^5 + 1.25S^4T + 2.5S^3T^2 + 2.5S^2T^3 + 1.25ST^4 + 0.25T^5\big)M + \big(- P^2R^2S^2 + 2P^2R^2ST - P^2R^2T^2 + P^2RS^3 - P^2RS^2T - P^2RST^2 + P^2RT^3 - 0.25P^2S^4 + 0.5P^2S^2T^2 - 0.25P^2T^4 + PR^2S^3 - PR^2S^2T - PR^2ST^2 + PR^2T^3 - PRS^4 + 2PRS^2T^2 - PRT^4 + 0.25PS^5 + 0.25PS^4T - 0.5PS^3T^2 - 0.5PS^2T^3 + 0.25PST^4 + 0.25PT^5 - 0.25R^2S^4 + 0.5R^2S^2T^2 - 0.25R^2T^4 + 0.25RS^5 + 0.25RS^4T - 0.5RS^3T^2 - 0.5RS^2T^3 + 0.25RST^4 + 0.25RT^5 - 0.0625S^6 - 0.125S^5T + 0.0625S^4T^2 + 0.25S^3T^3 + 0.0625S^2T^4 - 0.125ST^5 - 0.0625T^6\big) \\
    4\bigg(-\big(R + P\big)^3 - \big(P - S\big)^3 + \big(T - P\big)^3 - \big(R - S\big)^3 + \big(T - R\big)^3 + 2P^3 + 2R^3 + 8PRS + 8PRT - 8PST - 8RST - 2T^3 + \big(T + S\big)^3\bigg)M^3 + 2\big(4P^3R + 8P^2R^2 - 8P^2RS - 8P^2RT - P^2S^2 - 2P^2ST - P^2T^2 + 4PR^3 - 8PR^2S - 8PR^2T + 2PRS^2 + 4PRST + 2PRT^2 + 2PS^3 + 6PS^2T + 6PST^2 + 2PT^3 - R^2S^2 - 2R^2ST - R^2T^2 + 2RS^3 + 6RS^2T + 6RST^2 + 2RT^3 - S^4 - 4S^3T - 6S^2T^2 - 4ST^3 - T^4\big)M^2 + \big(- 4P^3R^2 - 4P^2R^3 + 4P^2R^2S + 4P^2R^2T + 2P^2RS^2 + 4P^2RST + 2P^2RT^2 + 2PR^2S^2 + 4PR^2ST + 2PR^2T^2 - 2PRS^3 - 6PRS^2T - 6PRST^2 - 2PRT^3 - 0.25PS^4 -PS^3T - 1.5PS^2T^2 - PST^3 - 0.25PT^4 - 0.25RS^4 - RS^3T - 1.5RS^2T^2 - RST^3 - 0.25RT^4 + 0.25S^5 + 1.25S^4T + 2.5S^3T^2 + 2.5S^2T^3 + 1.25ST^4 + 0.25T^5\big)M + \big(- P^2R^2S^2 + 2P^2R^2ST - P^2R^2T^2 + P^2RS^3 - P^2RS^2T - P^2RST^2 + P^2RT^3 - 0.25P^2S^4 + 0.5P^2S^2T^2 - 0.25P^2T^4 + PR^2S^3 - PR^2S^2T - PR^2ST^2 + PR^2T^3 - PRS^4 + 2PRS^2T^2 - PRT^4 + 0.25PS^5 + 0.25PS^4T - 0.5PS^3T^2 - 0.5PS^2T^3 + 0.25PST^4 + 0.25PT^5 - 0.25R^2S^4 + 0.5R^2S^2T^2 - 0.25R^2T^4 + 0.25RS^5 + 0.25RS^4T - 0.5RS^3T^2 - 0.5RS^2T^3 + 0.25RST^4 + 0.25RT^5 - 0.0625S^6 - 0.125S^5T + 0.0625S^4T^2 + 0.25S^3T^3 + 0.0625S^2T^4 - 0.125ST^5 - 0.0625T^6\big) \\
\end{align}

In [ ]:
(S - R)(S^2 - 2SR + R^2) = S^3 - 3RS^2 + 3SR^2 - R^3

In [158]:
sym.factor(-P**3 - 3*P**2*R + 3*P**2*S + 3*P**2*T - 3*P*R**2 + 8*P*R*S + 8*P*R*T - 3*P*S**2 - 8*P*S*T - 3*P*T**2 - R**3 + 3*R**2*S + 3*R**2*T - 3*R*S**2 - 8*R*S*T - 3*R*T**2 + S**3 + 3*S**2*T + 3*S*T**2 + T**3)

-P**3 - 3*P**2*R + 3*P**2*S + 3*P**2*T - 3*P*R**2 + 8*P*R*S + 8*P*R*T - 3*P*S**2 - 8*P*S*T - 3*P*T**2 - R**3 + 3*R**2*S + 3*R**2*T - 3*R*S**2 - 8*R*S*T - 3*R*T**2 + S**3 + 3*S**2*T + 3*S*T**2 + T**3

In [164]:
r1, r2, r3 = sym.solve(numerator, M)

In [ ]:
sym.cancel(sym.together(r1))

In [149]:
denominator

(0.5*M*P + 0.5*M*R - 0.5*M*S - 0.5*M*T - 0.5*P*R + 0.125*S**2 + 0.25*S*T + 0.125*T**2)**3

Denominator will be negative if and only if...

\begin{align}
    % 4MP + 4MR - 4MS - 4MT - 4PR + S^2 + 2ST + T^2 < 0 \\
    % 4M\big((R + P) - (T + S)\big) - 4PR + (T + S)^2< 0 \\
    % 4M\big((R + P) - (T + S)\big) < 4RP - (T + S)^2 \\
    M < \frac{1}{4}\frac{4RP - (T + S)^2}{(R + P) - (T + S)} \\
    M < \frac{RP - \left(\frac{T + S}{2}\right)^2}{(R + P) - (T + S)} \\
\end{align}

...since we are looking at interior equilibrium the RHS of the above inequality will be positive.

In [139]:
_f = sym.lambdify((T, R, P, S, M), evaluated_e3, modules = "numpy")

def plot_non_invadable_by_randomista(T, R, P, S):
    fig, ax = plt.subplots(1, 1, figsize=(10,6))
    Ms = np.linspace(0, min(0.5 * (T + S), P), 100)
    ax.plot(Ms, _f(T, R, P, S, Ms))
    ax.set_xlabel(r"$M$", fontsize=15)
    ax.set_ylabel(r"$e_3$", rotation="horizontal", fontsize=15)
    plt.show()

In [140]:
# sliders used to control the Prisoner's Dilemma Payoffs
T_slider = widgets.FloatSlider(value=10, min=0, max=100, step=0.1, description=r"$T$")
R_slider = widgets.FloatSlider(value=8, min=0, max=100, step=0.1, description=r"$R$")
P_slider = widgets.FloatSlider(value=6, min=0, max=100, step=0.1, description=r"$P$")
S_slider = widgets.FloatSlider(value=4, min=0, max=100, step=0.1, description=r"$S$")

w = widgets.interactive(plot_non_invadable_by_randomista, T=T_slider, R=R_slider, P=P_slider, S=S_slider)
display(w)

A Jupyter Widget

In [157]:
solutions = sym.solve(simplified_e3, M)

KeyboardInterrupt: 